# ssd resnet50 v1 Object Detection API and AWS Sagemaker

In this notebook, we are using ssd resnet 50 from tf object detection api 

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
arn:aws:iam::477094387272:role/service-role/AmazonSageMaker-ExecutionRole-20230910T073521


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
        'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://sab-object-detection-udacity/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the the repository
* build the docker image and push it 
* print the container name

In [6]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


In [8]:
# build and push the docker image. This code can be commented after being ran once.
# This will take around 10 mins.
image_name = 'tf2-object-detection-ssd-resnet50'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection-ssd-resnet50
Sending build context to Docker daemon    729MB
Step 1/17 : FROM tensorflow/tensorflow:2.12.0-gpu
2.12.0-gpu: Pulling from tensorflow/tensorflow

f31133a9: Pulling fs layer 
d51e1731: Pulling fs layer 
8ea3b9cc: Pulling fs layer 
a321f215: Pulling fs layer 
e49da50d: Pulling fs layer 
e5648755: Pulling fs layer 
ccdbe31d: Pulling fs layer 
01a2ab97: Pulling fs layer 
5d720362: Pulling fs layer 
75b4be3d: Pulling fs layer 
407e661a: Pulling fs layer 
48faa02b: Pulling fs layer 
99bd4375: Pulling fs layer 
6f8491b9: Pulling fs layer 
1da7cf83: Pulling fs layer 
a3817946: Pulling fs layer 
086451f7: Pulling fs layer 


75b4be3d: Extracting  2.215GB/2.522GBBDownloading  484.6MB/2.522GB

59c27a44: Pull complete 134kB/1.134kBDigest: sha256:ce32fe3c53f94938de1513b3fe5fa25aa655a2f4c4bced42cf598fb8ce911219
Status: Downloaded newer image for tensorflow/tensorflow:2.12.0-gpu
 ---> cbe3a4f4c2a0
Step 2/17 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Running in 79c66d7d0a6b
Removing intermediate container 79c66d7d0a6b
 ---> 99c68e86502d
Step 3/17 : RUN rm /etc/apt/sources.list.d/cuda.list
 ---> Running in 2bf1d9163834
Removing intermediate container 2bf1d9163834
 ---> 1333b768eb2d
Step 4/17 : RUN apt-key del 7fa2af80
 ---> Running in 661d411609ef
OK
Removing intermediate container 661d411609ef
 ---> aeb07e59e57e
Step 5/17 : RUN apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
 ---> Running in dd2296ff3e34
Executing: /tmp/apt-key-gpghome.eZe7Ceh0sU/gpg.1.sh --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/3bf863cc.pub
gpg: requesting key from 'https://developer.download.nvidia.co

The following NEW packages will be installed:
  blt fontconfig-config fonts-dejavu-core git git-man less libbsd0 libcairo2
  libcbor0.6 libcurl3-gnutls libdrm-amdgpu1 libdrm-common libdrm-intel1
  libdrm-nouveau2 libdrm-radeon1 libdrm2 libedit2 libegl-dev libegl-mesa0
  libegl1 liberror-perl libfido2-1 libfontconfig1 libfreetype6 libgbm1
  libgdk-pixbuf2.0-0 libgdk-pixbuf2.0-bin libgdk-pixbuf2.0-common libgl-dev
  libgl1 libgl1-mesa-dev libgl1-mesa-dri libglapi-mesa libgles-dev libgles1
  libgles2 libglvnd-dev libglvnd0 libglx-dev libglx-mesa0 libglx0
  libimagequant0 libjbig0 libjpeg-turbo8 libjpeg8 liblcms2-2 libllvm12
  libopengl-dev libopengl0 libpciaccess0 libpixman-1-0 libpng16-16
  libprotobuf-dev libprotobuf-lite17 libprotobuf17 libprotoc17
  libpthread-stubs0-dev libsensors-config libsensors5 libtcl8.6 libtiff5
  libtk8.6 libvulkan1 libwayland-client0 libwayland-server0 libwebp6
  libwebpdemux2 libwebpmux3 libx11-6 libx11-data libx11-dev libx11-xcb1
  libxau-dev libxau6 libxcb

Get:65 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors-config all 1:3.6.0-2ubuntu1.1 [6052 B]
Get:66 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libsensors5 amd64 1:3.6.0-2ubuntu1.1 [27.2 kB]
Get:67 http://archive.ubuntu.com/ubuntu focal/main amd64 libvulkan1 amd64 1.2.131.2-1 [93.3 kB]
Get:68 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1-mesa-dri amd64 21.2.6-0ubuntu0.1~20.04.2 [11.0 MB]
Get:69 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx-mesa0 amd64 21.2.6-0ubuntu0.1~20.04.2 [137 kB]
Get:70 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libglx0 amd64 1.3.2-1~ubuntu0.20.04.2 [32.5 kB]
Get:71 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 libgl1 amd64 1.3.2-1~ubuntu0.20.04.2 [85.8 kB]
Get:72 http://archive.ubuntu.com/ubuntu focal/main amd64 xorg-sgml-doctools all 1:1.11-1 [12.9 kB]
Get:73 http://archive.ubuntu.com/ubuntu focal/main amd64 x11proto-dev all 2019.2-1ubuntu1 [594 kB]
Get:74 http:

...

      Successfully uninstalled tensorflow-io-gcs-filesystem-0.31.0
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.22.1
    Uninstalling protobuf-4.22.1:
      Successfully uninstalled protobuf-4.22.1
  Attempting uninstall: pillow
    Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: requests
    Found existing installation: requests 2.22.0
    Uninstalling requests-2.22.0:
      Successfully uninstalled requests-2.22.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oau

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.2/249.2 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 618.5/618.5 kB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 70.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 17.5 MB/s eta 0:00:00
  Created wheel for sagemaker-training: filename=sagemaker_training-4.7.0-cp38-cp38-linux_x86_64.whl size=93403 sha256=7e00d851d94c5e39a989cec4c0081faa29ba4e1ddc955b5fb56da157620034da
  Stored in directory: /root/.cache/pip/wheels/16/76/fb/32f317f8d2ea470479580d8d5dcccc1bfd7994b273df803aaf
  Created wheel for inotify_simple: filename=inotify_simple-1.2.1-py3-non

4a3e387: Pushing  662.6MB/3.084GBPushing  208.7MB/3.084GBPushing  426.6MB/3.084GB

4a3e387: Pushing  2.362GB/3.084GBPushing  857.8MB/3.084GBPushing  2.255GB/3.084GB

4a3e387: Pushed   3.117GB/3.084GB20231001095149: digest: sha256:dcc5123718becedba112cec831d702d0985bb8524eb17ea23b73b124c35c4a09 size: 6605
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [12]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

477094387272.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection-ssd-resnet50:20231001095149


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be ajusted if you were to experiment with other architectures.

In [6]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/ssd-resnet50-v1.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/ssd-resnet50-v1.tar.gz --strip-components 2 --directory source_dir/checkpoint ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint

--2023-10-01 09:27:27--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 172.253.115.207, 172.253.122.207, 142.251.163.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|172.253.115.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘/tmp/ssd-resnet50-v1.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 6.99M 33s
    50K .......... .......... .......... .......... ..........  0% 23.2M 22s
   100K .......... .......... .......... .......... ..........  0% 22.6M 18s
   150K .......... .......... .......... .......... ..........  0% 25.0M 16s
   200K .......... .......... .......... .......... ..........  0% 34.0M 14s
   250K .......... .......... .......... .......... ..........  0% 20.4M 14s
   300K .......... .......... .......... ......

  5100K .......... .......... .......... .......... ..........  2% 92.3M 6s
  5150K .......... .......... .......... .......... ..........  2% 54.4M 6s
  5200K .......... .......... .......... .......... ..........  2%  232M 6s
  5250K .......... .......... .......... .......... ..........  2% 38.8M 6s
  5300K .......... .......... .......... .......... ..........  2% 66.4M 6s
  5350K .......... .......... .......... .......... ..........  2% 65.2M 6s
  5400K .......... .......... .......... .......... ..........  2% 23.6M 6s
  5450K .......... .......... .......... .......... ..........  2%  145M 6s
  5500K .......... .......... .......... .......... ..........  2%  332M 6s
  5550K .......... .......... .......... .......... ..........  2% 12.8M 6s
  5600K .......... .......... .......... .......... ..........  2%  341M 6s
  5650K .......... .......... .......... .......... ..........  2%  333M 6s
  5700K .......... .......... .......... .......... ..........  2%  386M 6s
  5750K ....

 10500K .......... .......... .......... .......... ..........  4%  241M 7s
 10550K .......... .......... .......... .......... ..........  4%  104M 7s
 10600K .......... .......... .......... .......... ..........  4%  122M 7s
 10650K .......... .......... .......... .......... ..........  4% 91.0M 7s
 10700K .......... .......... .......... .......... ..........  4% 32.0M 7s
 10750K .......... .......... .......... .......... ..........  4%  135M 7s
 10800K .......... .......... .......... .......... ..........  4%  321M 7s
 10850K .......... .......... .......... .......... ..........  4%  362M 7s
 10900K .......... .......... .......... .......... ..........  4% 22.6M 7s
 10950K .......... .......... .......... .......... ..........  4%  200M 7s
 11000K .......... .......... .......... .......... ..........  4%  365M 7s
 11050K .......... .......... .......... .......... ..........  4% 24.1M 7s
 11100K .......... .......... .......... .......... ..........  4%  328M 7s
 11150K ....

 15900K .......... .......... .......... .......... ..........  6%  339M 5s
 15950K .......... .......... .......... .......... ..........  6%  203M 5s
 16000K .......... .......... .......... .......... ..........  6%  341M 5s
 16050K .......... .......... .......... .......... ..........  6%  270M 5s
 16100K .......... .......... .......... .......... ..........  6%  364M 5s
 16150K .......... .......... .......... .......... ..........  6%  379M 5s
 16200K .......... .......... .......... .......... ..........  6%  364M 5s
 16250K .......... .......... .......... .......... ..........  6%  303M 5s
 16300K .......... .......... .......... .......... ..........  6%  347M 5s
 16350K .......... .......... .......... .......... ..........  6%  747K 6s
 16400K .......... .......... .......... .......... ..........  6%  278M 6s
 16450K .......... .......... .......... .......... ..........  6%  253M 6s
 16500K .......... .......... .......... .......... ..........  6%  110M 6s
 16550K ....

 21300K .......... .......... .......... .......... ..........  8%  346M 5s
 21350K .......... .......... .......... .......... ..........  8%  316M 5s
 21400K .......... .......... .......... .......... ..........  8%  335M 5s
 21450K .......... .......... .......... .......... ..........  8%  349M 5s
 21500K .......... .......... .......... .......... ..........  9%  369M 5s
 21550K .......... .......... .......... .......... ..........  9%  304M 5s
 21600K .......... .......... .......... .......... ..........  9%  356M 5s
 21650K .......... .......... .......... .......... ..........  9%  350M 5s
 21700K .......... .......... .......... .......... ..........  9%  303M 5s
 21750K .......... .......... .......... .......... ..........  9%  316M 5s
 21800K .......... .......... .......... .......... ..........  9%  339M 5s
 21850K .......... .......... .......... .......... ..........  9%  322M 5s
 21900K .......... .......... .......... .......... ..........  9%  378M 5s
 21950K ....

...

221100K .......... .......... .......... .......... .......... 92%  182M 0s
221150K .......... .......... .......... .......... .......... 92% 3.80M 0s
221200K .......... .......... .......... .......... .......... 92% 84.4M 0s
221250K .......... .......... .......... .......... .......... 92%  117M 0s
221300K .......... .......... .......... .......... .......... 92% 5.30M 0s
221350K .......... .......... .......... .......... .......... 92%  263M 0s
221400K .......... .......... .......... .......... .......... 92%  301M 0s
221450K .......... .......... .......... .......... .......... 92%  248M 0s
221500K .......... .......... .......... .......... .......... 92%  370M 0s
221550K .......... .......... .......... .......... .......... 92%  174M 0s
221600K .......... .......... .......... .......... .......... 92%  320M 0s
221650K .......... .......... .......... .......... .......... 92% 9.29M 0s
221700K .......... .......... .......... .......... .......... 92%  315M 0s
221750K ....

226500K .......... .......... .......... .......... .......... 94%  391M 0s
226550K .......... .......... .......... .......... .......... 94%  377M 0s
226600K .......... .......... .......... .......... .......... 94%  300M 0s
226650K .......... .......... .......... .......... .......... 94%  365M 0s
226700K .......... .......... .......... .......... .......... 94%  383M 0s
226750K .......... .......... .......... .......... .......... 94%  390M 0s
226800K .......... .......... .......... .......... .......... 94%  337M 0s
226850K .......... .......... .......... .......... .......... 94%  247M 0s
226900K .......... .......... .......... .......... .......... 94%  388M 0s
226950K .......... .......... .......... .......... .......... 94%  252M 0s
227000K .......... .......... .......... .......... .......... 94%  286M 0s
227050K .......... .......... .......... .......... .......... 94%  352M 0s
227100K .......... .......... .......... .......... .......... 95%  390M 0s
227150K ....

231900K .......... .......... .......... .......... .......... 97%  348M 0s
231950K .......... .......... .......... .......... .......... 97%  372M 0s
232000K .......... .......... .......... .......... .......... 97%  332M 0s
232050K .......... .......... .......... .......... .......... 97%  362M 0s
232100K .......... .......... .......... .......... .......... 97%  381M 0s
232150K .......... .......... .......... .......... .......... 97%  224M 0s
232200K .......... .......... .......... .......... .......... 97%  193M 0s
232250K .......... .......... .......... .......... .......... 97% 11.3M 0s
232300K .......... .......... .......... .......... .......... 97%  265M 0s
232350K .......... .......... .......... .......... .......... 97%  190M 0s
232400K .......... .......... .......... .......... .......... 97%  270M 0s
232450K .......... .......... .......... .......... .......... 97%  316M 0s
232500K .......... .......... .......... .......... .......... 97%  157M 0s
232550K ....

237300K .......... .......... .......... .......... .......... 99%  210M 0s
237350K .......... .......... .......... .......... .......... 99%  377M 0s
237400K .......... .......... .......... .......... .......... 99%  306M 0s
237450K .......... .......... .......... .......... .......... 99%  390M 0s
237500K .......... .......... .......... .......... .......... 99%  372M 0s
237550K .......... .......... .......... .......... .......... 99%  298M 0s
237600K .......... .......... .......... .......... .......... 99%  330M 0s
237650K .......... .......... .......... .......... .......... 99%  230M 0s
237700K .......... .......... .......... .......... .......... 99%  318M 0s
237750K .......... .......... .......... .......... .......... 99%  352M 0s
237800K .......... .......... .......... .......... .......... 99%  322M 0s
237850K .......... .......... .......... .......... .......... 99%  341M 0s
237900K .......... .......... .......... .......... .......... 99%  366M 0s
237950K ....

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [13]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir":"/opt/training",        
        "pipeline_config_path": "pipeline.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.p3.2xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection-resnet50'
)

estimator.fit(inputs)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: tf2-object-detection-resnet50-2023-10-01-10-43-39-316


2023-10-01 10:43:50 Starting - Starting the training job...
2023-10-01 10:44:15 Starting - Preparing the instances for training.........
2023-10-01 10:45:23 Downloading - Downloading input data...
2023-10-01 10:45:58 Training - Downloading the training image............
2023-10-01 10:48:09 Training - Training image download completed. Training in progress....2023-10-01 10:48:37,433 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-01 10:48:37,468 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-01 10:48:37,502 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2023-10-01 10:48:37,515 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "curre

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W1001 10:48:51.951128 140278790854464 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W1001 10:48:54.966875 140278790854464 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W1001 10:48:56.538476 140278

INFO:tensorflow:Step 300 per-step time 0.366s
I1001 10:51:55.519712 140278790854464 model_lib_v2.py:705] Step 300 per-step time 0.366s
INFO:tensorflow:{'Loss/classification_loss': 0.21995051,
 'Loss/localization_loss': 0.35163856,
 'Loss/regularization_loss': 0.2568097,
 'Loss/total_loss': 0.82839876,
 'learning_rate': 0.01733305}
I1001 10:51:55.520071 140278790854464 model_lib_v2.py:708] {'Loss/classification_loss': 0.21995051,
 'Loss/localization_loss': 0.35163856,
 'Loss/regularization_loss': 0.2568097,
 'Loss/total_loss': 0.82839876,
 'learning_rate': 0.01733305}
INFO:tensorflow:Step 400 per-step time 0.367s
I1001 10:52:32.188888 140278790854464 model_lib_v2.py:705] Step 400 per-step time 0.367s
INFO:tensorflow:{'Loss/classification_loss': 0.22584881,
 'Loss/localization_loss': 0.27865508,
 'Loss/regularization_loss': 0.25712377,
 'Loss/total_loss': 0.7616277,
 'learning_rate': 0.0186664}
I1001 10:52:32.189175 140278790854464 model_lib_v2.py:708] {'Loss/classification_loss': 0.2258

INFO:tensorflow:Step 1700 per-step time 0.367s
I1001 11:00:30.805203 140278790854464 model_lib_v2.py:705] Step 1700 per-step time 0.367s
INFO:tensorflow:{'Loss/classification_loss': 0.15671249,
 'Loss/localization_loss': 0.17784087,
 'Loss/regularization_loss': 0.2587476,
 'Loss/total_loss': 0.593301,
 'learning_rate': 0.03599995}
I1001 11:00:30.805533 140278790854464 model_lib_v2.py:708] {'Loss/classification_loss': 0.15671249,
 'Loss/localization_loss': 0.17784087,
 'Loss/regularization_loss': 0.2587476,
 'Loss/total_loss': 0.593301,
 'learning_rate': 0.03599995}
INFO:tensorflow:Step 1800 per-step time 0.367s
I1001 11:01:07.495791 140278790854464 model_lib_v2.py:705] Step 1800 per-step time 0.367s
INFO:tensorflow:{'Loss/classification_loss': 0.20452192,
 'Loss/localization_loss': 0.24591435,
 'Loss/regularization_loss': 0.25879905,
 'Loss/total_loss': 0.7092353,
 'learning_rate': 0.037333302}
I1001 11:01:07.496141 140278790854464 model_lib_v2.py:708] {'Loss/classification_loss': 0.20

I1001 11:02:57.039264 140478404777792 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use `tf.cast` instead.
W1001 11:03:02.588323 140478404777792 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Finished eval step 0
I1001 11:03:02.618988 140478404777792 model_lib_v2.py:966] Finished eval step 0
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well a

I1001 11:07:56.449031 140254839441216 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1001 11:08:06.581729 140254839441216 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1001 11:08:09.998592 140254839441216 signature_serialization.py:148] Function `call_func` contains input name(s) resource with unsupported characters which will be renamed to weightsharedconvolutionalboxpredictor_classpredictiontower_conv2d_3_batchnorm_feature_4_fusedbatchnormv3_readvariableop_1_resource in the SavedModel.
I1001 11:08:11.513298 140254839441216 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
W1001 11:08:14.275866 140254839441216 save_impl.py:66] Skipping full serialization of Keras layer <object_detection.meta_architectures.ssd_meta_arch.SSDMetaArch object at 0x7f8ed849da30>, because it is not built.
I1001 11:08:29.556078 140254839441216 save.py:274] Found untraced functions such as We

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the intial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the writeup.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your writeup), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

In [ ]:
# your writeup goes here.